This notebook produces visualizations of the model interpretable parameters (or parameter–derived quantities) in space and time.

In [1]:
import jax
jax.config.update("jax_enable_x64", True)

In [ ]:
from itertools import chain
import os
from pathlib import Path

import dask
import diffrax as dfx
import equinox as eqx
import jax.numpy as jnp
import jax.random as jrd
from jaxtyping import Float, PyTree, Key, Real
from hydra_zen import instantiate, load_from_yaml
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
import xarray as xr

from pastax.gridded import Gridded
from pastax.simulator import DeterministicSimulator
from pastax.trajectory import Trajectory, TrajectoryEnsemble
from pastax.utils import meters_to_degrees

from src.commons.dataset import TrajectoryDataset, worker_init_fn
from src.ec_mlp.data_driven_model import DataDrivenModel as StochDataDrivenModel
from src.ec_mlp.drift_model import DriftModel as StochDriftModel
from src.ec_mlp.trainer_module import TrainerModule as StochTrainerModule
from src.ls_mlp.data_driven_model import DataDrivenModel as DeterDataDrivenModel
from src.ls_mlp.drift_model import DriftModel as DeterStochModel
from src.ls_mlp.trainer_module import TrainerModule as DeterTrainerModule

In [3]:
ROOT = Path(".")

STOCH_RUNS_DIR = ROOT / "euler_criterion/mlp/multirun/2026-02-08/09-28-33"
STOCH_RUN_ID = "15"
stoch_run_dir = STOCH_RUNS_DIR / STOCH_RUN_ID

DETER_RUN_DIR = ROOT / "least_squares/mlp/outputs/2026-02-17/15-00-33"
deter_run_dir = DETER_RUN_DIR

DATA_DIR = ROOT / "data"

In [4]:
from_datetime_str = "1994-06-01"
to_datetime_str = "2025-08-01"

In [5]:
batch_size = 32
torch_n_workers = 12
multiprocessing_context = "forkserver" if torch_n_workers > 0 else None
prefetch_factor = 2 if torch_n_workers > 0 else 0
persistent_workers = False
in_order = False

dask_num_workers = 4

In [6]:
delta_t = jnp.asarray(5 * 60.0)  # 5 minutes in seconds, should be small enough w.r.t. to the diffusion magnitude
sim_days = 6
n_steps = int((sim_days * 24 * 60 * 60 / delta_t))

n_samples = 100

## Load best models

In [7]:
stoch_run_cfg = load_from_yaml(stoch_run_dir / ".hydra" / "config.yaml")
deter_run_cfg = load_from_yaml(deter_run_dir / ".hydra" / "config.yaml")

In [8]:
trunk = instantiate(stoch_run_cfg.trunk)()
physical_head = instantiate(stoch_run_cfg.physical_head)()
mdn_head = instantiate(stoch_run_cfg.mdn_head)()

stoch_data_driven_model = StochDataDrivenModel(trunk, physical_head, mdn_head)
stoch_drift_model = StochDriftModel(
    data_driven_model=stoch_data_driven_model, 
    stress_normalization=1.0, 
    wind_normalization=1.0,
    delta_t=1.0 * 60.0 * 60.0
)

stoch_drift_model = StochTrainerModule.load_from_checkpoint(
    stoch_run_dir / "best_model.ckpt", drift_model=stoch_drift_model
).drift_model

/home/bertrava/.local/share/mamba/envs/pastax_global_calibration/lib/python3.11/site-packages/lightning/pytorch/core/saving.py:96: The state dict in PosixPath('euler_criterion/mlp/multirun/2026-02-08/09-28-33/15/best_model.ckpt') contains no parameters.


In [9]:
mlp = instantiate(deter_run_cfg.mlp)()

deter_data_driven_model = DeterDataDrivenModel(mlp)
deter_drift_model = DeterStochModel(
    data_driven_model=deter_data_driven_model, 
    stress_normalization=1.0, 
    wind_normalization=1.0
)

deter_drift_model = DeterTrainerModule.load_from_checkpoint(
    deter_run_dir / "best_model.ckpt", drift_model=deter_drift_model
).drift_model

/home/bertrava/.local/share/mamba/envs/pastax_global_calibration/lib/python3.11/site-packages/lightning/pytorch/core/saving.py:96: The state dict in PosixPath('least_squares/mlp/outputs/2026-02-17/15-00-33/best_model.ckpt') contains no parameters.


## Reference trajectories

In [10]:
traj_ds = xr.open_zarr(DATA_DIR / f"gdp_interp_clean_{from_datetime_str}_{to_datetime_str}_test_traj.zarr")

In [11]:
overlap = (
    (traj_ds["time"][:, 0] < np.datetime64("2008-01-01")) & (traj_ds["time"][:, -1] > np.datetime64("2008-01-01"))
)

traj_ds = traj_ds.where(~overlap.compute(), drop=True)

In [ ]:
out_store = DATA_DIR / f"traj_errors_{from_datetime_str}_{to_datetime_str}.zarr"

if not out_store.exists():
    ts_empty_arr = np.full((traj_ds.traj.size, traj_ds.obs.size), np.nan, dtype=np.float32)
    ens_ts_empty_arr = np.full((traj_ds.traj.size, n_samples, traj_ds.obs.size), np.nan, dtype=np.float32)

    output_template = xr.Dataset(
        {
            "reference_time": (("traj", "obs"), ts_empty_arr),
            "reference_lat": (("traj", "obs"), ts_empty_arr),
            "reference_lon": (("traj", "obs"), ts_empty_arr),
            "simulated_ensemble_lat": (("traj", "sample", "obs"), ens_ts_empty_arr),
            "simulated_ensemble_lon": (("traj", "sample", "obs"), ens_ts_empty_arr),
            "simulated_mode_lat": (("traj", "obs"), ts_empty_arr),
            "simulated_mode_lon": (("traj", "obs"), ts_empty_arr),
            "simulated_mean_lat": (("traj", "obs"), ts_empty_arr),
            "simulated_mean_lon": (("traj", "obs"), ts_empty_arr),
            "ensemble_separation_distance": (("traj", "sample", "obs"), ens_ts_empty_arr),
            "ensemble_liu_index": (("traj", "sample", "obs"), ens_ts_empty_arr),
            "ensemble_separation_distance_crps": (("traj", "obs"), ts_empty_arr),
            "ensemble_liu_index_crps": (("traj", "obs"), ts_empty_arr),
            "mode_separation_distance": (("traj", "obs"), ts_empty_arr),
            "mode_liu_index": (("traj", "obs"), ts_empty_arr),
            "mean_separation_distance": (("traj", "obs"), ts_empty_arr),
            "mean_liu_index": (("traj", "obs"), ts_empty_arr),
        },
        coords={
            "traj": traj_ds.traj.values,
            "obs": traj_ds.obs.values,
            "sample": np.arange(n_samples)
        }
    ).chunk(
        {"traj": batch_size, "obs": -1, "sample": -1}
    )

    output_template.to_zarr(out_store, compute=False, consolidated=False)
    
    i = 0
else:
    output_ds = xr.open_zarr(out_store)
    output_ds.to_zarr(
        DATA_DIR / f"traj_errors_{from_datetime_str}_{to_datetime_str}.zarr.bck", consolidated=False, mode="w"
    )
    
    t0 = output_ds["reference_time"].isel(obs=0).astype("datetime64[s]")
    lat0 = output_ds["reference_lat"].isel(obs=0)
    lon0 = output_ds["reference_lon"].isel(obs=0)

    t0_mask = traj_ds["time"].isel(obs=0).isin(np.unique(t0.dropna("traj")))
    lat0_mask = traj_ds["lat"].isel(obs=0).isin(np.unique(lat0.dropna("traj")))
    lon0_mask = traj_ds["lon"].isel(obs=0).isin(np.unique(lon0.dropna("traj")))

    traj_ds = traj_ds.where(t0_mask & lat0_mask & lon0_mask, drop=True)

    t0_mask = np.isnan(t0)
    lat0_mask = np.isnan(lat0)
    lon0_mask = np.isnan(lon0)
    output_mask = t0_mask | lat0_mask | lon0_mask

    output_ds.sortby(output_mask).to_zarr(
        DATA_DIR / f"traj_errors_{from_datetime_str}_{to_datetime_str}.zarr.tmp", consolidated=False, mode="w"
    )
    os.replace(DATA_DIR / f"traj_errors_{from_datetime_str}_{to_datetime_str}.zarr.tmp", out_store)

    i = int((~output_mask).sum())  # this way we're sure to start at a multiple of the batch size

In [13]:
traj_coarse_ds = traj_ds.where((traj_ds["time"] < np.datetime64("2008-01-01")).compute(), drop=True)
traj_fine_ds = traj_ds.where((traj_ds["time"] >= np.datetime64("2008-01-01")).compute(), drop=True)

In [14]:
traj_coarse_path = DATA_DIR / f"gdp_interp_clean_{from_datetime_str}_{to_datetime_str}_test_traj_coarse.zarr"
traj_fine_path = DATA_DIR / f"gdp_interp_clean_{from_datetime_str}_{to_datetime_str}_test_traj_fine.zarr"

In [15]:
traj_coarse_ds.to_zarr(traj_coarse_path, mode="w")
traj_fine_ds.to_zarr(traj_fine_path, mode="w")

/home/bertrava/.local/share/mamba/envs/pastax_global_calibration/lib/python3.11/site-packages/zarr/api/asynchronous.py:247: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


/home/bertrava/.local/share/mamba/envs/pastax_global_calibration/lib/python3.11/site-packages/zarr/api/asynchronous.py:247: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


## Forcings

In [16]:
duacs_path = (
    DATA_DIR / f"cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.125deg_P1D_{from_datetime_str}_{to_datetime_str}.zarr"
)
era5_coarse_path = DATA_DIR / f"cmems_obs-wind_glo_phy_my_l4_0.25deg_PT1H_{from_datetime_str}_2008-01-01.zarr"
era5_fine_path = DATA_DIR / f"cmems_obs-wind_glo_phy_my_l4_0.125deg_PT1H_2008-01-01_{to_datetime_str}.zarr"

## Dataloading

In [17]:
torch_dataset_coarse = None
torch_dataset_fine = None

if traj_coarse_ds.traj.size > 0:
    torch_dataset_coarse = TrajectoryDataset(traj_coarse_path, duacs_path, era5_coarse_path)
if traj_fine_ds.traj.size > 0:
    torch_dataset_fine = TrajectoryDataset(traj_fine_path, duacs_path, era5_fine_path)

In [18]:
torch_dataloader_coarse = None
torch_dataloader_fine = None

if torch_dataset_coarse is not None:
    torch_dataloader_coarse = DataLoader(
        torch_dataset_coarse,
        batch_size=batch_size,
        num_workers=torch_n_workers,
        worker_init_fn=worker_init_fn,
        multiprocessing_context=multiprocessing_context,
        prefetch_factor=prefetch_factor,
        persistent_workers=persistent_workers,
        in_order=in_order
    )
if torch_dataset_fine is not None:
    torch_dataloader_fine = DataLoader(
        torch_dataset_fine,
        batch_size=batch_size,
        num_workers=torch_n_workers,
        worker_init_fn=worker_init_fn,
        multiprocessing_context=multiprocessing_context,
        prefetch_factor=prefetch_factor,
        persistent_workers=persistent_workers,
        in_order=in_order
    )

In [19]:
dask.config.set(scheduler="threads", num_workers=dask_num_workers)

In [20]:
torch_dataloader = None
total_steps = 0

if torch_dataloader_coarse is not None and torch_dataloader_fine is not None:
    torch_dataloader = chain(torch_dataloader_coarse, torch_dataloader_fine)
    total_steps = len(torch_dataloader_coarse) + len(torch_dataloader_fine)
elif torch_dataloader_coarse is not None:
    torch_dataloader = torch_dataloader_coarse
    total_steps = len(torch_dataloader_coarse)
elif torch_dataloader_fine is not None:
    torch_dataloader = torch_dataloader_fine
    total_steps = len(torch_dataloader_fine)

## Drift integration

In [21]:
def to_jax(pytree: PyTree) -> PyTree:
    return jax.tree.map(jnp.asarray, pytree)


@eqx.filter_jit  # this improves performances
def to_trajectory(
    traj_lat: Float[jax.Array, "batch traj_length"], 
    traj_lon: Float[jax.Array, "batch traj_length"], 
    traj_time: Float[jax.Array, "batch traj_length"]
) -> Trajectory:
    trajectories = eqx.filter_vmap(lambda _latlon, _time: Trajectory.from_array(values=_latlon, times=_time))(
        jnp.stack((traj_lat, traj_lon), axis=-1), traj_time
    )

    return trajectories


@eqx.filter_jit  # this improves performances
def to_gridded(
    dict_vars: dict[str, Float[jax.Array, "T N N"]], 
    time: Float[jax.Array, "T"], 
    lat: Float[jax.Array, "N"], 
    lon: Float[jax.Array, "N"]
) -> Gridded:
    gridded = eqx.filter_vmap(Gridded.from_array)(
        dict_vars, time, lat, lon
    )

    return gridded

In [22]:
def month_from_epoch(seconds):
    # Days since Unix epoch (1970-01-01)
    days = seconds // 86400

    # Convert Unix epoch days to civil date where a year starts on March 1st
    # Unix epoch 1970-01-01 corresponds to civil date offset 719468
    z = days + 719468

    # The leap-year pattern repeats every 400 years (146097 days), one era corresponds to one such period
    era = (z >= 0).astype(jnp.int64) * z // 146097 + (z < 0).astype(jnp.int64) * ((z - 146096) // 146097)
    doe = z - era * 146097
    yoe = (doe - doe // 1460 + doe // 36524 - doe // 146096) // 365
    doy = doe - (365 * yoe + yoe // 4 - yoe // 100)
    
    # The month is shifted back so that January is month 1
    mp = (5 * doy + 2) // 153
    month = mp + 3 - 12 * (mp // 10)

    return month


def stoch_dynamics(
    t: Real[jax.Array, ""], 
    x: Float[jax.Array, "2"], 
    args: tuple[StochDriftModel, Float[jax.Array, ""], Gridded, Gridded, Real[jax.Array, ""], Key[jax.Array, "n_steps"]]
) -> Float[jax.Array, "2"]:
    lat, lon = x
    drift_model, delta_t, duacs, era5, t0, keys = args

    key = keys[((t - t0) / delta_t).astype(int)]

    duacs_forcing = duacs.interp("ugos", "vgos", time=t, latitude=lat, longitude=lon)
    era5_forcing = era5.interp(
        "eastward_stress", "northward_stress", "eastward_wind", "northward_wind", time=t, latitude=lat, longitude=lon
    )

    ugos, vgos = duacs_forcing["ugos"], duacs_forcing["vgos"]
    eastward_stress, northward_stress = era5_forcing["eastward_stress"], era5_forcing["northward_stress"]
    eastward_wind, northward_wind = era5_forcing["eastward_wind"], era5_forcing["northward_wind"]

    u, v = drift_model.sample_velocity(
        ugos, vgos, eastward_stress, northward_stress, eastward_wind, northward_wind, 
        month_from_epoch(t), lat, lon,
        key, delta_t
    )

    vu = jnp.stack((v, u), axis=-1)
    vu = meters_to_degrees(vu, latitude=lat)

    return vu


def mode_dynamics(
    t: Real[jax.Array, ""], 
    x: Float[jax.Array, "2"], 
    args: tuple[StochDriftModel, Gridded, Gridded]
) -> Float[jax.Array, "2"]:
    lat, lon = x
    drift_model, duacs, era5 = args

    duacs_forcing = duacs.interp("ugos", "vgos", time=t, latitude=lat, longitude=lon)
    era5_forcing = era5.interp(
        "eastward_stress", "northward_stress", "eastward_wind", "northward_wind", time=t, latitude=lat, longitude=lon
    )

    ugos, vgos = duacs_forcing["ugos"], duacs_forcing["vgos"]
    eastward_stress, northward_stress = era5_forcing["eastward_stress"], era5_forcing["northward_stress"]
    eastward_wind, northward_wind = era5_forcing["eastward_wind"], era5_forcing["northward_wind"]

    u, v = drift_model.estimate_mode_velocity(
        ugos, vgos, eastward_stress, northward_stress, eastward_wind, northward_wind, 
        month_from_epoch(t), lat, lon
    )

    vu = jnp.stack((v, u), axis=-1)
    vu = meters_to_degrees(vu, latitude=lat)

    return vu


def deter_dynamics(
    t: Real[jax.Array, ""], 
    x: Float[jax.Array, "2"], 
    args: tuple[DeterStochModel, Gridded, Gridded]
) -> Float[jax.Array, "2"]:
    lat, lon = x
    drift_model, duacs, era5 = args

    duacs_forcing = duacs.interp("ugos", "vgos", time=t, latitude=lat, longitude=lon)
    era5_forcing = era5.interp(
        "eastward_stress", "northward_stress", "eastward_wind", "northward_wind", time=t, latitude=lat, longitude=lon
    )

    ugos, vgos = duacs_forcing["ugos"], duacs_forcing["vgos"]
    eastward_stress, northward_stress = era5_forcing["eastward_stress"], era5_forcing["northward_stress"]
    eastward_wind, northward_wind = era5_forcing["eastward_wind"], era5_forcing["northward_wind"]

    uv = drift_model.estimate_velocity(
        ugos, vgos, eastward_stress, northward_stress, eastward_wind, northward_wind, 
        month_from_epoch(t), lat, lon
    )

    vu = jnp.stack((uv.imag, uv.real), axis=-1)
    vu = meters_to_degrees(vu, latitude=lat)

    return vu

In [23]:
simulator = DeterministicSimulator()

In [24]:
def simulate(
    trajectory: Trajectory, 
    duacs: Gridded, 
    era5: Gridded, 
    key: Key[jax.Array, ""]
) -> tuple[Trajectory, Trajectory, Trajectory]:
    x0 = trajectory.origin
    ts = trajectory.times.value

    solver_cls = dfx.Euler  # corresponds to our training criterion
    saveat = lambda: dfx.SaveAt(ts=ts)
    step_size_controller = lambda: dfx.StepTo(jnp.linspace(ts[0], ts[-1], n_steps + 1))

    sample_keys = jax.random.split(key, n_samples)

    stoch_args = (stoch_drift_model, delta_t, duacs, era5, ts[0])
    mode_args = (stoch_drift_model, duacs, era5)
    deter_args = (deter_drift_model, duacs, era5)

    stoch_sol = jax.vmap(
        lambda sample_key: simulator(
            dynamics=stoch_dynamics,
            args=(*stoch_args, jax.random.split(sample_key, n_steps)),
            x0=x0,
            ts=ts,
            solver=solver_cls(),
            saveat=saveat(),
            stepsize_controller=step_size_controller(),
            max_steps=n_steps,
        )
    )(sample_keys)

    mode_sol = simulator(
        dynamics=mode_dynamics,
        args=mode_args,
        x0=x0,
        ts=ts,
        solver=solver_cls(),
        saveat=saveat(),
        stepsize_controller=step_size_controller()
    )

    deter_sol = simulator(
        dynamics=deter_dynamics,
        args=deter_args,
        x0=x0,
        ts=ts,
        solver=solver_cls(),
        saveat=saveat(),
        stepsize_controller=step_size_controller()
    )

    return stoch_sol, mode_sol, deter_sol

In [ ]:
def to_numpy(arr: Float[jax.Array, "..."]) -> Float[np.ndarray, "..."]:
    return np.asarray(arr, dtype=np.float32)


key = jrd.key(0)
j = 0

for trajectory_batch, duacs_batch, era5_batch in tqdm(torch_dataloader, total=total_steps):
    print(f"Processing batch {j} / {total_steps}...")

    trajectory_batch = to_jax(trajectory_batch)
    duacs_batch = to_jax(duacs_batch)
    era5_batch = to_jax(era5_batch)

    batch_size = trajectory_batch[0].shape[0]

    trajectory_batch = to_trajectory(*trajectory_batch)
    duacs_batch = to_gridded(*duacs_batch)
    era5_batch = to_gridded(*era5_batch)
    
    key, subkey = jrd.split(key, 2)
    keys = jax.random.split(subkey, batch_size)

    print("Simulating trajectories...")

    stoch_ens_batch, mode_est_batch, deter_est_batch = eqx.filter_vmap(simulate)(
        trajectory_batch, duacs_batch, era5_batch, keys
    )

    stoch_ens_batch = eqx.filter_vmap(
        lambda traj: TrajectoryEnsemble.from_array(values=traj.locations.value, times=traj.times.value[0])
    )(stoch_ens_batch)

    print("Computing metrics...")

    stoch_sep_dist_crps = eqx.filter_vmap(
        lambda stoch_ens, ref: stoch_ens.crps(ref, metric_func=Trajectory.separation_distance)
    )(
        stoch_ens_batch, trajectory_batch
    )

    stoch_liu_index_crps = eqx.filter_vmap(
        lambda stoch_ens, ref: stoch_ens.crps(ref, metric_func=Trajectory.liu_index)
    )(
        stoch_ens_batch, trajectory_batch
    )

    stoch_sep_dist = eqx.filter_vmap(
        lambda stoch_ens, ref: stoch_ens.map(lambda est: ref.separation_distance(est))
    )(
        stoch_ens_batch, trajectory_batch
    )

    stoch_liu_index = eqx.filter_vmap(
        lambda stoch_ens, ref: stoch_ens.map(lambda est: ref.liu_index(est))
    )(
        stoch_ens_batch, trajectory_batch
    )

    mode_sep_dist = eqx.filter_vmap(lambda mode_est, ref: ref.separation_distance(mode_est))(
        mode_est_batch, trajectory_batch
    )

    mode_liu_index = eqx.filter_vmap(lambda mode_est, ref: ref.liu_index(mode_est))(
        mode_est_batch, trajectory_batch
    )

    deter_sep_dist = eqx.filter_vmap(lambda deter_est, ref: ref.separation_distance(deter_est))(
        deter_est_batch, trajectory_batch
    )

    deter_liu_index = eqx.filter_vmap(lambda deter_est, ref: ref.liu_index(deter_est))(
        deter_est_batch, trajectory_batch
    )
    
    print("Saving results...")

    ds = xr.Dataset(
        {
            "reference_time": (("traj", "obs"), np.asarray(trajectory_batch.times.value, dtype="datetime64[s]")),
            "reference_lat": (("traj", "obs"), to_numpy(trajectory_batch.locations.value[:, :, 0])),
            "reference_lon": (("traj", "obs"), to_numpy(trajectory_batch.locations.value[:, :, 1])),
            "simulated_ensemble_lat": (
                ("traj", "sample", "obs"), to_numpy(stoch_ens_batch.locations.value[:, :, :, 0])
            ),
            "simulated_ensemble_lon": (
                ("traj", "sample", "obs"), to_numpy(stoch_ens_batch.locations.value[:, :, :, 1])
            ),
            "simulated_mode_lat": (("traj", "obs"), to_numpy(mode_est_batch.locations.value[:, :, 0])),
            "simulated_mode_lon": (("traj", "obs"), to_numpy(mode_est_batch.locations.value[:, :, 1])),
            "simulated_mean_lat": (("traj", "obs"), to_numpy(deter_est_batch.locations.value[:, :, 0])),
            "simulated_mean_lon": (("traj", "obs"), to_numpy(deter_est_batch.locations.value[:, :, 1])),
            "ensemble_separation_distance": (("traj", "sample", "obs"), to_numpy(stoch_sep_dist.value)),
            "ensemble_liu_index": (("traj", "sample", "obs"), to_numpy(stoch_liu_index.value)),
            "ensemble_separation_distance_crps": (("traj", "obs"), to_numpy(stoch_sep_dist_crps.value)),
            "ensemble_liu_index_crps": (("traj", "obs"), to_numpy(stoch_liu_index_crps.value)),
            "mode_separation_distance": (("traj", "obs"), to_numpy(mode_sep_dist.value)),
            "mode_liu_index": (("traj", "obs"), to_numpy(mode_liu_index.value)),
            "mean_separation_distance": (("traj", "obs"), to_numpy(deter_sep_dist.value)),
            "mean_liu_index": (("traj", "obs"), to_numpy(deter_liu_index.value)),
        },
        coords={
            "traj": traj_ds.traj.values[i: i + batch_size],
            "obs": traj_ds.obs.values,
            "sample": np.arange(n_samples)
        }
    )

    region = dict(traj=slice(i, i + batch_size), obs="auto", sample="auto")

    while True:
        try:
            ds.to_zarr(out_store, region=region, consolidated=False)
        except Exception as e:
            print(f"Error while saving batch {j} / {total_steps}: {e}. Retrying...")
        else:
            break

    i += batch_size
    j += 1

    print(f"Batch {j} / {total_steps} done.")


  0%|                                                                                               | 0/1190 [00:00<?, ?it/s]

Processing batch 0 / 1190...


Simulating trajectories...


Computing metrics...


Saving results...


...

Saving results...



 19%|███████████████▎                                                                | 227/1190 [2:55:54<11:09:45, 41.73s/it]

Batch 227 / 1190 done.


Processing batch 227 / 1190...
Simulating trajectories...


Computing metrics...
